In [ ]:
import cx_Oracle
import pandas as pd
import numpy as np
import sys
import os
import importlib
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
from time import time
from plotly.subplots import make_subplots
from datastory import DataStory

from google.cloud import secretmanager
from google.cloud.bigquery import Client, LoadJobConfig

sys.path.append('/home/jupyter/pensjon-data-analyse/lib')
import pandas_utils
import pesys_utils
import autograd_convenience
import utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(autograd_convenience)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
tuning = 1000

con = pesys_utils.open_pen_connection()
df_bq = pandas_utils.pandas_from_sql('/home/jupyter/pensjon-data-analyse/sql/laaste_vedtak.sql', con=con, tuning=tuning, lowercase=True)
con.close()

In [ ]:
df_bq.head()

In [ ]:
def df_to_bq(project_id, full_table_id, dataframe, write_disposition):
    client = Client(project="pensjon-saksbehandli-prod-1f83")
    job_config = LoadJobConfig(write_disposition=write_disposition)
    
    start = time()
    job = client.load_table_from_dataframe(dataframe, full_table_id, job_config=job_config)
    job.result()

    end = time()

    print(f'{len(dataframe)} rad(er) ble skrevet til bigquery etter {end-start} sekunder.')

In [ ]:
df_to_bq(
    project_id='pensjon-saksbehandli-prod-1f83',
    full_table_id='pensjon-saksbehandli-prod-1f83.vedtak.laast_data_handling',
    dataframe=df_bq,
    write_disposition='WRITE_TRUNCATE'        
)